In [1]:
import pandas as pd

# Caminho do arquivo
caminho_arquivo = r'BDados_FISCREP_2022.xlsx'

# Lendo o arquivo em um DataFrame
df = pd.read_excel(caminho_arquivo)

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df.head(2)

,Source.Name,Entidade,Entidade - Unidade,Patrol Type,Nome do meio naval,Tipo (inspection or sighting),GDH (AAAAMMDD),GDH (hh:mm),N.º de missão,N.º único de inspeção (nr sequencial/ano),Tipo de alvo,Estado do registo do objeto (ISO 3166-1 alpha-3),Registo nacional de identificação,CFR,Nome,Atividade do Alvo,Arte de pesca,tamanho da malhagem,Área FAO,Área geográfico DGRM,Latitude,Longitude,Porto (não aplicável se inspeção for no mar),Local (não aplicável se inspeção for no mar),Espécie 1 - sigla,Espécie 1 - Qtd. KG,Espécie 1 - NB-INDX,Espécie 2 - sigla,Espécie 2 - Qtd. KG,Espécie 2 - NB-INDX,Espécie 3 - sigla,Espécie 3 - Qtd. KG,Espécie 3 - NB-INDX,Espécie 4 - sigla,Espécie 4 - Qtd. KG,Espécie 4 - NB-INDX,Espécie 5 - sigla,Espécie 5 - Qtd. KG,Espécie 5 - NB-INDX,Espécie 6 - sigla,Espécie 6 - Qtd. KG,Espécie 6 - NB-INDX,Espécie 7 - sigla,Espécie 7 - Qtd. KG,Espécie 7 - NB-INDX,Espécie 8 - sigla,Espécie 8 - Qtd. KG,Espécie 8 - NB-INDX,Espécie 9 - sigla,Espécie 9 - Qtd. KG,Espécie 9 - NB-INDX,Espécie 10 - sigla,Espécie 10 - Qtd. KG,Espécie 10 - NB-INDX,Espécie 11 - sigla,Espécie 11 - Qtd. KG,Espécie 11 - NB-INDX,Espécie 12 - sigla,Espécie 12 - Qtd. KG,Espécie 12 - NB-INDX,Resultado (âmbito CN),Resultado (âmbito DGRM - pesca),Infração 0,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,Observações,IMO number,Local de registo (EU vessel fleet),MMSI,Vessel Type (EU vessel fleet),Main fishing Gear,Secondary fishing Gear 1,Secondary fishing Gear 2,LOA,LBP,Tonnage GT,Other tonnage,Segment,Power of main engine,Power of auxiliary engine,Hull material,Year of construction,Age of inspection,Latitude (decimal),Longitude (decimal)
0,Pescado_SADAP_2010.xlsx,CN,LAGAO,Sea,N.R.P. DRAGAO,Inspection,2010-01-03,09:00:00,NaN,LAGAO-1/2010,PCM-Embarcação,UNK,O-2003-C,NaN,SANTA MARIA DOS ANJOS,ANC - Ancorado,GTR,100MM,27.9.a-Portuguese Waters / East,Continente,N3700.80,W00803.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Legal,Legal,XIV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.013333,-8.053333
1,Pescado_SADAP_2010.xlsx,CN,PACINE,Sea,N.R.P. CACINE,Inspection,2010-01-03,17:10:00,NaN,PACINE-002/2010,PCM-Embarcação,UNK,VC-167-L,NaN,JORGE MARIA,FIS - fishing,GTR,MAIOR IGUAL 100,27.9.a-Portuguese Waters / East,Continente,N4119.2,W00857.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Legal,Legal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.320000,-8.958333


In [4]:
# Convertendo a coluna "GDH" para o formato de data
df['GDH (AAAAMMDD)'] = pd.to_datetime(df['GDH (AAAAMMDD)']).dt.strftime('%m/%d/%Y')

In [5]:
df['GDH (AAAAMMDD)'] = pd.to_datetime(df['GDH (AAAAMMDD)'])

In [6]:
df = df.sort_values(by=['Nome', 'GDH (AAAAMMDD)'])

In [7]:
df = df.reset_index(drop=True)

In [8]:
# Criando uma nova coluna "Categoria" preenchida com valores em branco
df['Categoria'] = ''
# Contar a ocorrência de cada valor em 'Nome'
count_dict = df['Nome'].value_counts().to_dict()

# Dicionário para controlar se a primeira linha já foi encontrada para um determinado 'Nome'
first_row_dict = {}

# Iterar sobre as linhas do DataFrame
for index, row in df.iterrows():
    nome = row['Nome']
    if pd.notnull(nome):
        if count_dict[nome] > 1:
            if nome not in first_row_dict:
                first_row_dict[nome] = index
            if index == first_row_dict[nome]:
                df.at[index, 'Categoria'] = '1ª Fiscalização'
            else:
                if row['Resultado (âmbito CN)'] == 'Presumível infrator' and df.loc[index - 1, 'Resultado (âmbito CN)'] == 'Legal':
                    df.at[index, 'Categoria'] = 'Type 3'
                elif row['Resultado (âmbito CN)'] == 'Legal' and df.loc[index - 1, 'Resultado (âmbito CN)'] == 'Presumível infrator':
                    df.at[index, 'Categoria'] = 'Type 2'
                elif row['Resultado (âmbito CN)'] == 'Presumível infrator' and df.loc[index - 1, 'Resultado (âmbito CN)'] == 'Presumível infrator':
                    df.at[index, 'Categoria'] = 'Type 4'
                elif row['Resultado (âmbito CN)'] == 'Legal' and df.loc[index - 1, 'Resultado (âmbito CN)'] == 'Legal':
                    df.at[index, 'Categoria'] = 'Type 1'
        else:
            df.at[index, 'Categoria'] = 'Única Fiscalização'



In [9]:
df[["Nome", "GDH (AAAAMMDD)", "Resultado (âmbito CN)","Categoria"]].iloc[4020:4050]

,Nome,GDH (AAAAMMDD),Resultado (âmbito CN),Categoria
4020,CORINNE,2016-08-30,Legal,Type 1
4021,CORINNE,2017-03-29,Legal,Type 1
4022,CORINNE,2017-07-30,Legal,Type 1
4023,CORINNE,2018-11-01,Legal,Type 1
4024,CORINNE,2019-06-15,Legal,Type 1
4025,CORINNE,2020-06-10,Legal,Type 1
4026,CORINNE I,2016-06-25,Legal,1ª Fiscalização
4027,CORINNE I,2018-08-04,Legal,Type 1
4028,CORINNE I,2018-10-05,Legal,Type 1
4029,CORRECIO,2012-10-27,Legal,Única Fiscalização


In [10]:
df[df["Nome"] == "COSTA SUL"][["Nome", "GDH (AAAAMMDD)", "Resultado (âmbito CN)", "Categoria"]]


,Nome,GDH (AAAAMMDD),Resultado (âmbito CN),Categoria
4174,COSTA SUL,2010-03-23,Presumível infrator,1ª Fiscalização
4175,COSTA SUL,2010-08-01,Presumível infrator,Type 4
4176,COSTA SUL,2010-08-25,Legal,Type 2
4177,COSTA SUL,2010-10-09,Presumível infrator,Type 3
4178,COSTA SUL,2010-10-17,Presumível infrator,Type 4
4179,COSTA SUL,2012-08-03,Presumível infrator,Type 4
4180,COSTA SUL,2012-09-20,Legal,Type 2
4181,COSTA SUL,2012-11-20,Legal,Type 1
4182,COSTA SUL,2012-12-28,Legal,Type 1
4183,COSTA SUL,2014-03-14,Presumível infrator,Type 3


In [11]:
# Exportar o DataFrame para um arquivo CSV
df.to_excel(r'dataset_cat1.xlsx', index=False)